## CNN

In [ ]:
from CNN.mnist_model import ConvNet
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms
import quartz
from quartz import layers
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
model = ConvNet(10)
model.load_state_dict(torch.load("CNN/mnist-convnet.pth", map_location=torch.device('cpu')))
model.eval()

class Lenet1(nn.Module):
            def __init__(self):
                super(Lenet1, self).__init__()
                self.features = nn.Sequential(
                    model.conv1,
                    nn.ReLU(),
                    model.pool1,
                    model.conv2,
                    nn.ReLU(),
                    model.pool2,
                    #model.drop2,
                    model.conv3,
                    nn.ReLU(),
#                     #model.drop3,
#                     nn.Flatten(),
#                     model.fc1,
#                     nn.ReLU(),
                )
            def forward(self, x):
                x = self.features(x)
                return x

layer = Lenet1()
layer.eval()

In [ ]:
def get_weights_biases(model):
    parameters = list(model.parameters())
    weights = [weight.detach().numpy().copy() for weight in parameters[::2][::2]]
    biases = [bias.detach().numpy().copy() for bias in parameters[1::2][::2]]
    return weights, biases

weights, biases = get_weights_biases(model)

In [ ]:
([weight.max() for weight in weights])

In [ ]:
([weight.min() for weight in weights])

In [ ]:
# weights[1] /= 1.5
# biases[1] /= 1.5
# weights[2] /= 1.6
# biases[2] /= 1.6
# weights[3] /= 3
# biases[3] /= 3

In [ ]:
layer.features

In [ ]:
# with torch.no_grad():
#     layer.features[3].weight /= 1.5
#     layer.features[3].bias /= 1.5
#     layer.features[6].weight /= 1.6
#     layer.features[6].bias /= 1.6
#     layer.features[9].weight /= 3
#     layer.features[9].bias /= 3

## build model

In [ ]:
t_max = 2**6
input_dims = (1,28,28)
pool_kernel_size = [2,2]
batch_size = 1

loihi_model = quartz.Network(t_max, [
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights[0], biases=biases[0]),
    layers.MaxPool2D(kernel_size=pool_kernel_size),
    layers.Conv2D(weights=weights[1], biases=biases[1]),
    layers.MaxPool2D(kernel_size=pool_kernel_size),
    layers.Conv2D(weights=weights[2], biases=biases[2]),
#     layers.Dense(weights=weights[3], biases=biases[3], rectifying=True),
])

## load data

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),])
test_set = datasets.MNIST('./CNN/data', train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
samples, targets = next(iter(test_loader))

In [ ]:
# # loihi_model.layers[1].blocks[0]
# [len(block.connections) for block in loihi_model.layers[3].blocks if len(block.connections) > 0]

In [ ]:
loihi_output = loihi_model(samples.detach().numpy())

In [ ]:
layer_output = layer(samples)

In [ ]:
# loihi_output

## check equality

In [ ]:
assert len(loihi_output.flatten()) == len(layer_output.flatten())

In [ ]:
combinations = list(zip(loihi_output.flatten(), layer_output.detach().cpu().numpy().flatten()))

In [ ]:
combinations

In [ ]:
%debug

In [ ]:
targets